In [1]:
import os, sys
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFE

from numpy import set_printoptions
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB



# 3. Tiền xử lí dữ liệu

In [2]:
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/feature1"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
data=pd.read_excel(f'{exps_dir}/data/data_EDA.xlsx')
data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,...,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,...,Saab,92x,2000-2010,Y,17,10,2014,25,1,2015
1,Y,Middle,IN,250/500,2000,1197.22,5000000,MALE,MD,machine-op-inspct,...,Mercedes,E400,2000-2010,Y,27,6,2006,21,1,2015
2,Y,Young,OH,100/300,2000,1413.14,5000000,FEMALE,PhD,sales,...,Dodge,RAM,2000-2010,N,6,9,2000,22,2,2015
3,Y,Middle,IL,250/500,2000,1415.74,6000000,FEMALE,PhD,armed-forces,...,Chevrolet,Tahoe,2010++,Y,25,5,1990,10,1,2015
4,Y,Middle,IL,500/1000,1000,1583.91,6000000,MALE,Associate,sales,...,Accura,RSX,2010++,N,6,6,2014,17,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,N,Middle,OH,500/1000,1000,1310.80,0,FEMALE,Masters,craft-repair,...,Honda,Accord,2000-2010,N,16,7,1991,22,2,2015
996,Y,Middle,IL,100/300,1000,1436.79,0,FEMALE,PhD,prof-specialty,...,Volkswagen,Passat,2010++,N,5,1,2014,24,1,2015
997,Y,Young,OH,250/500,500,1383.49,3000000,FEMALE,Masters,armed-forces,...,Suburu,Impreza,1995-20001,N,17,2,2003,23,1,2015
998,Y,Old,IL,500/1000,2000,1356.92,5000000,MALE,Associate,handlers-cleaners,...,Audi,A5,1995-20001,N,18,11,2011,26,2,2015


In [4]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_excel(f'{exps_dir}/data/datatrain.xlsx', index=None)
data_test.to_excel(f'{exps_dir}/data/datatest.xlsx', index=None)

(670, 40) (330, 40)


In [5]:
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,...,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,...,Mercedes,ML350,2010++,Y,21,6,1994,3,1,2015
311,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,...,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
722,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,...,Suburu,Forrestor,2000-2010,Y,29,7,1996,16,1,2015
629,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,...,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,...,Saab,92x,2000-2010,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,...,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
270,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,...,Toyota,Highlander,2000-2010,N,15,4,2013,27,1,2015
860,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,...,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
435,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,...,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [6]:
numeric_columns=dict(np.load(f'{exps_dir}/data/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

array(['policy_annual_premium', 'capital-gains', 'capital-loss',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim', 'day_policy_bind_date', 'month_policy_bind_date',
       'year_policy_bind_date', 'day_incident_date',
       'month_incident_date', 'year_incident_date'], dtype='<U22')

In [7]:
category_columns=dict(np.load(f'{exps_dir}/data/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

array(['months_as_customer', 'age', 'policy_state', 'policy_csl',
       'policy_deductable', 'umbrella_limit', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage',
       'bodily_injuries', 'witnesses', 'police_report_available',
       'auto_make', 'auto_model', 'auto_year', 'fraud_reported'],
      dtype='<U27')

In [8]:
outliers_list=dict(np.load(f'{exps_dir}/data/outliers.npz'))['outliers']
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Áp dụng hàm cho các cột chứa dữ liệu cần loại bỏ ngoại lệ
for column in outliers_list:
    data_train = remove_outliers_iqr(data_train, column)
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,...,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,...,Mercedes,ML350,2010++,Y,21,6,1994,3,1,2015
311,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,...,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
722,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,...,Suburu,Forrestor,2000-2010,Y,29,7,1996,16,1,2015
629,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,...,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,...,Saab,92x,2000-2010,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,...,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
270,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,...,Toyota,Highlander,2000-2010,N,15,4,2013,27,1,2015
860,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,...,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
435,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,...,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [9]:
(data_train=='?').sum()

months_as_customer               0
age                              0
policy_state                     0
policy_csl                       0
policy_deductable                0
policy_annual_premium            0
umbrella_limit                   0
insured_sex                      0
insured_education_level          0
insured_occupation               0
insured_hobbies                  0
insured_relationship             0
capital-gains                    0
capital-loss                     0
incident_type                    0
collision_type                 112
incident_severity                0
authorities_contacted            0
incident_state                   0
incident_city                    0
incident_hour_of_the_day         0
number_of_vehicles_involved      0
property_damage                235
bodily_injuries                  0
witnesses                        0
police_report_available        207
total_claim_amount               0
injury_claim                     0
property_claim      

In [10]:
data_train['collision_type']=data_train['collision_type'].replace('?',np.nan)
data_train['collision_type'].fillna(method='ffill',inplace=True)
data_train[['property_damage','police_report_available']]=data_train[['property_damage','police_report_available']].replace('?','Unknow')
(data_train=='?').sum()

months_as_customer             0
age                            0
policy_state                   0
policy_csl                     0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
insured_sex                    0
insured_education_level        0
insured_occupation             0
insured_hobbies                0
insured_relationship           0
capital-gains                  0
capital-loss                   0
incident_type                  0
collision_type                 0
incident_severity              0
authorities_contacted          0
incident_state                 0
incident_city                  0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
property_damage                0
bodily_injuries                0
witnesses                      0
police_report_available        0
total_claim_amount             0
injury_claim                   0
property_claim                 0
vehicle_claim                  0
auto_make 

# 4.Chuẩn hóa dữ liệu

In [11]:
label_encoders = {}
for column in category_columns:
    label_encoder = LabelEncoder()
    data_train[column] = label_encoder.fit_transform(data_train[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = MinMaxScaler()
data_train[numeric_columns] = scaler.fit_transform(data_train[numeric_columns])
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,...,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,2,0,0,1,1,0.156407,0,1,2,11,...,8,24,2,1,0.666667,0.454545,0.166667,0.066667,0.0,0.0
311,0,2,1,0,2,0.116057,0,1,4,11,...,6,6,0,1,0.333333,0.454545,0.750000,0.500000,0.5,0.0
722,2,0,2,0,2,0.629083,0,0,0,5,...,11,15,1,1,0.933333,0.545455,0.250000,0.500000,0.0,0.0
629,2,0,0,0,2,0.403313,0,0,2,2,...,5,14,0,1,0.900000,0.363636,1.000000,0.000000,1.0,0.0
0,2,0,2,1,1,0.617128,0,1,4,2,...,10,1,1,1,0.533333,0.818182,1.000000,0.800000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2,1,2,2,1,0.397140,0,0,0,9,...,10,3,0,1,0.200000,0.818182,0.000000,0.400000,0.0,0.0
270,2,1,2,1,2,0.759830,0,1,1,5,...,12,18,1,0,0.466667,0.272727,0.958333,0.866667,0.0,0.0
860,2,0,1,0,1,0.317237,0,0,4,0,...,0,23,0,0,0.500000,0.909091,0.583333,0.166667,0.5,0.0
435,0,2,2,1,0,0.358103,0,1,4,6,...,4,27,0,0,0.933333,0.636364,0.375000,0.466667,0.5,0.0


In [12]:
x_train=data_train.drop('fraud_reported',axis=1)
y_train=data_train['fraud_reported']


In [13]:
from imblearn.over_sampling import SMOTE
x_train, y_train = SMOTE().fit_resample(x_train, y_train)
x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [14]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 1.0, 1: 1.0}

In [15]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 1002
Fold 0: 
+ train_idx: [   0    1    2    3    4    5    6    7    8    9   11   12   13   14
   15   16   17   18   19   20   21   22   24   26   27   28   31   32
   33   34   35   36   37   38   40   41   42   43   45   46   47   48
   49   50   51   52   53   56   57   58   61   62   64   65   68   69
   71   73   74   75   77   79   80   81   82   83   84   85   87   89
   90   91   92   93   94   95   97   98   99  100  102  103  104  105
  106  108  109  111  112  113  114  115  116  117  118  119  121  122
  123  124  125  126  127  128  129  130  131  132  133  134  135  138
  140  141  142  143  144  145  146  147  148  149  150  151  152  153
  154  155  156  157  159  160  161  162  163  164  165  166  167  169
  170  171  172  173  175  176  177  178  179  180  181  182  183  185
  186  187  188  189  190  191  192  193  194  195  196  197  200  201
  202  203  204  205  206  207  211  212  214  216  217  219  220  222
  223  224  225  226  227  228  229  23

In [16]:
object_cols_test=dict(np.load(f'{exps_dir}/data/columns_dtype.npz',allow_pickle=True))['category_columns']
numeric_cols_test=dict(np.load(f'{exps_dir}/data/columns_dtype.npz',allow_pickle=True))['numeric_columns']
for column in outliers_list:
    data_test = remove_outliers_iqr(data_test, column)
data_test['collision_type'].replace('?',np.nan,inplace=True)
data_test['collision_type'].fillna(method='ffill',inplace=True)
data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
(data_test=='?').sum()
label_encoders = {}
for column in object_cols_test:
    label_encoder = LabelEncoder()
    data_test[column] = label_encoder.fit_transform(data_test[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = MinMaxScaler()
data_test[numeric_cols_test] = scaler.fit_transform(data_test[numeric_cols_test])

x_test=data_test.drop('fraud_reported',axis=1)
y_test=data_test['fraud_reported']
data_test.to_excel(f'{save_dir}/df_test_minmax.xlsx', index=False)
x_test.to_excel(f'{save_dir}/x_test.xlsx', index=False)
y_test.to_excel(f'{save_dir}/y_test.xlsx', index=False)